In [1]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install -U scikit-learn

In [2]:
# !pip install pandas
# !pip install matplotlib

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy 
import torch
from torch import nn
import sklearn

In [4]:
data_x = pd.read_csv('train.csv')
x = data_x.values
y = pd.read_csv('trainLabels.csv').values

x = torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.float)
x = torch.round(x , decimals=3)
# y = torch.round(y , decimals=3)
x.shape

torch.Size([999, 40])

In [7]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , shuffle=True)

In [30]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(40 , 20),
            nn.Tanh(),
            nn.Linear(20 , 20),
            nn.Tanh(),
            nn.Linear(20 , 1),
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        return self.sigmoid(self.layer1(x))

model = Model() 

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=0.1)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer ,step_size=500 ,  gamma = 0.1)
from sklearn.metrics import accuracy_score


In [ ]:
# torch.manual_seed(42)

epochs = 1000
test_acc_list = []
no_epoch = []
train_acc = []

for epoch in range(epochs):

  model.train()

  y_logits = model(x_train).squeeze()
  # y_logits = torch.squeeze(y_logit)
  y_preds = torch.round(y_logits)

  loss = loss_fn(y_logits , y_train.squeeze())
  acc = accuracy_score(y_true = y_train,
                       y_pred = y_preds.detach().numpy()
                       )
  train_acc.append(acc)
  optimizer.zero_grad()

  loss.backward() # does back propogation

  optimizer.step() # updates the parameters with the helpof gradient descent
  scheduler.step()
  model.eval()

  with torch.inference_mode():
    test_logits = model(x_test).squeeze()
    test_pred= torch.round(test_logits)

    test_loss = loss_fn(test_logits,
                        y_test.squeeze())
    test_acc = accuracy_score(y_true=y_test,
                        y_pred = test_pred.detach().numpy())
    test_acc_list.append(test_acc)
    no_epoch.append(epoch)

    if epoch%50 == 0:
    # print(f"Epoch: {epoch} | Loss: {loss} | Acc: {acc:.2f}% | Test Loss: {test_loss} | Test Accuracy: {test_acc:.2f}%")
      print(f"epoch {epoch}  ||  loss {loss:.4f}  || train accuracy {acc:.4f} ||  test_loss {test_loss:.4f}  ||  test_accuracy {test_acc:.4f}")

In [36]:
test = pd.read_csv('test.csv').values
import csv
test_data = torch.from_numpy(test).type(torch.float)
res = torch.round(model(test_data))

In [39]:
# filename = "university_records.csv"
l=[]
d = ['Id' , 'Solution']
# for i in range(res.shape[0]):
#     l.append(str(int(res[i].item())))
# writing to csv file
with open('result.csv', 'w' , newline='') as csvfile:
    # creating a csv dict writer object
    writer = csv.writer(csvfile)
    # writing headers (field names)
    writer.writerow(d)
    # val = torch.item(res)
    # writing data rows
    # writer.writerows(l)
    y_lo = torch.round(res)
    for i in range(res.shape[0]):
     writer.writerow([i+1 , int(y_lo[i].item())])

svc_out = torch.round(res)
svc_out.squeeze()
solution = pd.DataFrame({'Id': range(1, len(svc_out.squeeze()) + 1),
               'Solution': svc_out.type(torch.int64).detach().numpy().squeeze()})
solution.to_csv('solution.csv',index=False)

# Model 2 with Sklearn's Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

lg=LogisticRegression()
lg.fit(x_train, y_train.squeeze())
lg_out=lg.predict(x_test)
accuracy_score(y_test , lg_out)
